In [1]:
import pandas as pd
import numpy as np
import joblib  # พระเอกของเราในการ Export Model
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# ==========================================
# 1. Data Preparation (Simulation)
# ==========================================
print(">>> Step 1: Data Acquisition & Splitting")

# โหลดข้อมูล
data = load_breast_cancer()
X, y = data.data, data.target

# จำลองสถานการณ์: แกล้งทำข้อมูลให้แหว่งๆ (Missing Values) เพื่อโชว์ Data Prep
rng = np.random.RandomState(42)
n_samples = X.shape[0]
n_features = X.shape[1]
# สุ่มทำให้ข้อมูลหายไปบางจุด
X[rng.randint(0, n_samples, size=50), rng.randint(0, n_features, size=50)] = np.nan

# แบ่ง Train/Test (Test set คือของศักดิ์สิทธิ์ ห้ามแตะจนจบงาน)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training Data Shape: {X_train.shape}")
print(f"Test Data Shape:     {X_test.shape}")
print("-" * 30)

# ==========================================
# 2. Constructing the Full Pipeline
# ==========================================
print(">>> Step 2: Building the Pipeline")

# นี่คือหัวใจสำคัญ: เรามัดรวมทุกขั้นตอนเป็นก้อนเดียว
# 1. Imputer: เติมค่าที่หายไป (Data Prep)
# 2. Scaler: ปรับสเกลข้อมูล (Preprocessing)
# 3. Model: ตัวทำนาย (Modeling)

full_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # ถ้าเจอ NaN ให้เติมด้วย Median
    ('scaler', StandardScaler()),                   # ปรับ Standard Scale
    ('classifier', RandomForestClassifier(random_state=42)) # โมเดล
])

# ==========================================
# 3. Training & Tuning (Model Selection)
# ==========================================
print(">>> Step 3: Training & Hyperparameter Tuning")

# กำหนดสิ่งที่จะจูน
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5]
}

# ใช้ GridSearchCV เพื่อหาค่าที่ดีที่สุด (โดยใช้ Cross-Validation ในตัว)
grid_search = GridSearchCV(
    estimator=full_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# เริ่มสอนโมเดล (ขั้นตอนนี้กินเวลาสุด)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print(f"\nBest Parameters found: {grid_search.best_params_}")
print(f"Best CV Score: {grid_search.best_score_:.4f}")

# ==========================================
# 4. Final Evaluation
# ==========================================
print("-" * 30)
print(">>> Step 4: Final Evaluation on Test Set")

# Predict ข้อมูล Test (Pipeline จะจัดการ Impute -> Scale -> Predict ให้เองอัตโนมัติ)
y_pred = best_model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

# ==========================================
# 5. Export Model (The Deployment Step)
# ==========================================
print("-" * 30)
print(">>> Step 5: Exporting Model (Pickling)")

model_filename = 'breast_cancer_pipeline_v1.joblib'

# Save ไฟล์ .joblib (ข้างในมีทั้ง Logic การเติมค่า, การ Scale, และตัวโมเดล)
joblib.dump(best_model, model_filename)

print(f"SUCCESS: Model saved to '{model_filename}'")
print("พร้อมส่งไฟล์นี้ให้ทีม Dev/DE แล้วครับ!")

# ==========================================
# 6. Simulation: Usage in Production
# ==========================================
print("-" * 30)
print(">>> Step 6: Simulating Production Usage (Load & Predict)")

# สมมติว่านี่คือเครื่อง Server ของบริษัท หรือ API
# เราโหลดไฟล์ที่ Save ไว้ขึ้นมา
loaded_model = joblib.load(model_filename)

# สมมติมีข้อมูลลูกค้าใหม่เข้ามา 1 คน (New Data)
new_data = X_test[0].reshape(1, -1) # ข้อมูลจริงตัวแรก

# สั่ง Predict ได้เลย ไม่ต้องมานั่งเขียนโค้ด Impute หรือ Scale ใหม่
prediction = loaded_model.predict(new_data)
prob = loaded_model.predict_proba(new_data)

print(f"Input Data shape: {new_data.shape}")
print(f"Prediction Class: {prediction[0]}")
print(f"Probability: {prob[0]}")

>>> Step 1: Data Acquisition & Splitting
Training Data Shape: (455, 30)
Test Data Shape:     (114, 30)
------------------------------
>>> Step 2: Building the Pipeline
>>> Step 3: Training & Hyperparameter Tuning
Fitting 5 folds for each of 18 candidates, totalling 90 fits

Best Parameters found: {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200}
Best CV Score: 0.9626
------------------------------
>>> Step 4: Final Evaluation on Test Set
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.93      0.94        42
           1       0.96      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114

------------------------------
>>> Step 5: Exporting Model (Pickling)
SUCCESS: Model saved to 'breast_cancer_pipeline_v1.joblib'
พร้อมส่งไฟล์นี้ให้ทีม Dev/DE